In [1]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
import torch, gc
import os

In [2]:
# flair.device = torch.device('cuda')
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:40.00MiB"
torch.cuda.empty_cache()

In [3]:
columns = {0: 'text', 1: 'ner'}

In [4]:
data_folder = 'corpus'

In [5]:
corpus: Corpus = ColumnCorpus(data_folder, columns, train_file='train.txt', test_file='test.txt', dev_file='dev.txt')

2023-05-02 15:03:57,853 Reading data from corpus
2023-05-02 15:03:57,853 Train: corpus\train.txt
2023-05-02 15:03:57,853 Dev: corpus\dev.txt
2023-05-02 15:03:57,853 Test: corpus\test.txt


In [6]:
len(corpus.train)

2506

In [7]:
label_type = 'ner'

In [8]:
# torch.cuda.is_available()

In [9]:
label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)
print(label_dict)

2023-05-02 15:03:58,592 Computing label dictionary. Progress:


2506it [00:00, 80213.42it/s]

2023-05-02 15:03:58,639 Dictionary created for label 'ner' with 1 values: LOC (seen 368 times)
Dictionary with 1 tags: LOC


In [10]:
embeddings = TransformerWordEmbeddings(model='mrm8488/bert-spanish-cased-finetuned-ner', layers='-1', subtoken_pooling='first', fine_tune=True, use_context=True, model_max_length=512)

In [11]:
tagger = SequenceTagger(hidden_size=256, embeddings=embeddings, tag_dictionary=label_dict, tag_type = 'ner', use_crf=False, use_rnn=False, reproject_embeddings=False)

2023-05-02 15:04:02,736 SequenceTagger predicts: Dictionary with 5 tags: O, S-LOC, B-LOC, E-LOC, I-LOC


In [12]:
trainer = ModelTrainer(tagger, corpus)
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [13]:
gc.collect()
torch.cuda.empty_cache()
trainer.fine_tune('ner-roberta-fineTuning', learning_rate=5.0e-6, mini_batch_size=4, mini_batch_chunk_size=1)

2023-05-02 15:04:02,909 ----------------------------------------------------------------------------------------------------
2023-05-02 15:04:02,925 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(31003, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(

100%|██████████| 83/83 [00:03<00:00, 21.93it/s]

2023-05-02 15:06:03,063 Evaluating as a multi-label problem: False
2023-05-02 15:06:03,073 DEV : loss 0.005735316313803196 - f1-score (micro avg)  0.7826
2023-05-02 15:06:03,083 ----------------------------------------------------------------------------------------------------


2023-05-02 15:06:15,142 epoch 2 - iter 62/627 - loss 0.01914812 - time (sec): 12.06 - samples/sec: 532.71 - lr: 0.000005
2023-05-02 15:06:26,853 epoch 2 - iter 124/627 - loss 0.01744984 - time (sec): 23.77 - samples/sec: 543.27 - lr: 0.000005
2023-05-02 15:06:38,425 epoch 2 - iter 186/627 - loss 0.01468777 - time (sec): 35.34 - samples/sec: 556.95 - lr: 0.000005
2023-05-02 15:06:49,786 epoch 2 - iter 248/627 - loss 0.01270819 - time (sec): 46.70 - samples/sec: 540.69 - lr: 0.000005
2023-05-02 15:07:01,097 epoch 2 - iter 310/627 - loss 0.01209182 - time (sec): 58.01 - samples/sec: 551.59 - lr: 0.000005
2023-05-02 15:07:12,614 epoch 2 - iter 372/627 - loss 0.01132210 - time (sec): 69.53 - samples/sec: 555.29 - lr: 0.000005
2023-05-02 15:07:24,039 epoch 2 - iter 434/627 - loss 0.01185701 - time (sec): 80.96 - samples/sec: 545.52 - lr: 0.000005
2023-05-02 15:07:35,490 epoch 2 - iter 496/627 - loss 0.01135305 - time (sec): 92.41 - samples/sec: 544.70 - lr: 0.000005
2023-05-02 15:07:45,693 e

100%|██████████| 83/83 [00:04<00:00, 20.40it/s]

...

2023-05-02 15:22:40,967 ----------------------------------------------------------------------------------------------------
2023-05-02 15:22:40,967 Testing using last state of model ...


100%|██████████| 74/74 [00:04<00:00, 17.99it/s]

2023-05-02 15:22:45,081 Evaluating as a multi-label problem: False


2023-05-02 15:22:45,101 0.8704	0.94	0.9038	0.8246
2023-05-02 15:22:45,101 
Results:
- F-score (micro) 0.9038
- F-score (macro) 0.9038
- Accuracy 0.8246

By class:
              precision    recall  f1-score   support

         LOC     0.8704    0.9400    0.9038        50

   micro avg     0.8704    0.9400    0.9038        50
   macro avg     0.8704    0.9400    0.9038        50
weighted avg     0.8704    0.9400    0.9038        50

2023-05-02 15:22:45,101 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9038461538461539,
 'dev_score_history': [0.782608695652174,
  0.9047619047619048,
  0.923076923076923,
  0.9500000000000001,
  0.9500000000000001,
  0.9268292682926829,
  0.9500000000000001,
  0.9500000000000001,
  0.9500000000000001,
  0.9500000000000001],
 'train_loss_history': [0.5951989999721774,
  0.011289503440021411,
  0.005768010880937106,
  0.0035779869515810895,
  0.0021482238295143946,
  0.0009071132214562935,
  0.0011011203566378828,
  0.00046854101833760574,
  0.00022826923175538695,
  0.00026575377800824987],
 'dev_loss_history': [0.005735316313803196,
  0.005453998222947121,
  0.005980044137686491,
  0.005660683382302523,
  0.00532536068931222,
  0.005554931703954935,
  0.00590277835726738,
  0.005855642259120941,
  0.0060495734214782715,
  0.006015281192958355]}